# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'AQ8.KQ94.JT.KJ52'

auction = ["PAD_START", "PAD_START", "PAD_START", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '1N', 'insta_score': 0.998}]

In [4]:
auction = ["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': '4H', 'insta_score': 0.206, 'expected_score': 535, 'adjustment': 10},
 {'call': '4C', 'insta_score': 0.159, 'expected_score': 485, 'adjustment': 8},
 {'call': '3N', 'insta_score': 0.512, 'expected_score': 450, 'adjustment': 26}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['AQ8.KQ9x.JT.KJxx K9xx.Jxx.8xx.Qxx .A8xx.AQ9.AT98xx JTxxxx.Tx.Kxxxx. 0.77259 \n P-1N-P-3S-P-3N-P-4C-P-4H-P-5C-P-5H-P-7C-P-P-P (12.2)  \n P-1N-P-3S-P-4H-P-4N-P-5D-P-6C-P-P-P (12.2)  \n P-1N-P-3S-P-4C-P-4N-P-5D-P-6C-P-P-P (12.3) ',
 'AQ8.KQ9x.JT.KJxx JTxxx.JT8x.Kxx.x x.Axxx.AQx.AT8xx K9xx.x.98xxx.Q9x 0.75825 \n P-1N-P-3S-P-3N-P-P-P (12.2)  \n P-1N-P-3S-P-4H-P-4N-P-5D-P-6C-P-P-P (12.3)  \n P-1N-P-3S-P-4C-P-4N-P-5D-P-6C-P-P-P (12.2) ',
 'AQ8.KQ9x.JT.KJxx KT9x.JTxx.9xx.9x .A8xx.A8xx.AQxxx Jxxxxx.x.KQxx.T8 0.75648 \n P-1N-P-3S-P-3N-P-4C-P-4H-P-5C-P-5H-P-7C-P-P-P (12.2)  \n P-1N-P-3S-P-4H-P-4N-P-5D-P-6C-P-P-P (12.2)  \n P-1N-P-3S-P-4C-P-4N-P-5D-P-6C-P-P-P (12.2) ',
 'AQ8.KQ9x.JT.KJxx KJTxxx.8xx.8xx.x .AJTx.Axxx.A8xxx 9xxx.xx.KQ9x.QT9 0.74850 \n P-1N-P-3S-P-3N-P-4C-P-4H-P-5C-P-5H-P-6C-P-P-P (12.2)  \n P-1N-P-3S-P-4H-P-4N-P-5D-P-6C-P-P-P (12.2)  \n P-1N-P-3S-P-4C-P-4N-P-5D-P-6C-P-P-P (12.2) ',
 'AQ8.KQ9x.JT.KJxx JTxxx.x.Q9xxx.AT 9.AJ8x.AK8.Q98xx Kxxx.Txxx.xxx.xx 0.73873 \n P-1N-P-3S-P-3N-P-P-P

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

AQ8.KQ9x.JT.KJxx K9xx.Jxx.8xx.Qxx .A8xx.AQ9.AT98xx JTxxxx.Tx.Kxxxx.
AQ8.KQ9x.JT.KJxx JTxxx.JT8x.Kxx.x x.Axxx.AQx.AT8xx K9xx.x.98xxx.Q9x
AQ8.KQ9x.JT.KJxx KT9x.JTxx.9xx.9x .A8xx.A8xx.AQxxx Jxxxxx.x.KQxx.T8
AQ8.KQ9x.JT.KJxx KJTxxx.8xx.8xx.x .AJTx.Axxx.A8xxx 9xxx.xx.KQ9x.QT9
AQ8.KQ9x.JT.KJxx JTxxx.x.Q9xxx.AT 9.AJ8x.AK8.Q98xx Kxxx.Txxx.xxx.xx
AQ8.KQ9x.JT.KJxx KTxx.xx.Q9xxx.8x x.ATxx.A8x.AQT9x J9xxx.J8x.Kxx.xx
AQ8.KQ9x.JT.KJxx Txxx.JTx.AK9x.Tx .Axxx.Q8xx.AQ9xx KJ9xxx.8x.xxx.8x
AQ8.KQ9x.JT.KJxx K9xxxx.8.Q8xx.9x T.AJxx.Axx.AQT8x Jxx.Txxx.K9xx.xx
AQ8.KQ9x.JT.KJxx JT9xx.Jxxx.Q8x.x x.A8xx.AKx.QT98x Kxxx.T.9xxxx.Axx
AQ8.KQ9x.JT.KJxx KT9xxx.JT.Qxxxx. .Axxx.AK8x.AT8xx Jxxx.8xx.9x.Q9xx
AQ8.KQ9x.JT.KJxx KJTxx.Jxx.xxx.Qx .ATxx.AK8x.A98xx 9xxxx.8x.Q9xx.Tx
AQ8.KQ9x.JT.KJxx K9x.8x.Kxxxx.T8x x.AJxx.A9x.AQ9xx JTxxxx.Txx.Q8x.x
AQ8.KQ9x.JT.KJxx JTxx.x.Q9xx.AQ8x .AJ8x.AKxx.T9xxx K9xxxx.Txxx.8xx.
AQ8.KQ9x.JT.KJxx KJT9xx.xx.Q98xx. .AJxx.AKx.A9xxxx xxxx.T8x.xxx.QT8
AQ8.KQ9x.JT.KJxx KT9x.8xx.Kxxx.8x x.AJTx.A9x.AQT